In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import join
import csv
import sys
import jieba
import jieba.posseg as pseg
import re
import jieba.analyse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
print("【訊息】目前的工作目錄:",os.getcwd())
import math


【訊息】目前的工作目錄: /workspace/backup


In [2]:
csv.field_size_limit(int(sys.maxsize/10000000000))
df=pd.read_csv('./data/deal_final_0605', sep=',', engine='python', encoding='utf-8-sig')

利用正規表示法清理不必要的符號

In [3]:
itemName = df['item_name']
symbols = r"[！!\-(－（)）.（+/，#_;:=】？?《》|【\]\[~★♥~↘]"
tmp = [re.sub(symbols," ",str(title)) for title in itemName ]
symbols2 = r"[ \d]+元|[\d.]+折[起]?|\$[ \d]+"
itemNameC = [re.sub(symbols2," ",title) for title in tmp ]

利用Jieba套件對標題欄位進行斷詞

In [ ]:
itemNameTokenN = [ list(filter(lambda x: x!='' ,jieba.lcut(i))) for i in itemNameC ]
itemNameTokenN = [ list(filter(lambda x: x!=' ' ,i) ) for i in itemNameTokenN ]
itemNameTokenN

將斷詞結果與code統整成DataFrame

In [8]:
df_codetest = df.code.fillna(0)
df_codetest = np.int64(list(df_codetest))

In [9]:
df_codetest= np.array(df_codetest).reshape(-1,1)
itemNameTokenN = np.array(itemNameTokenN).reshape(-1,1)
tmp = np.concatenate((df_codetest,itemNameTokenN), axis=1)
code_token = pd.DataFrame(tmp,columns=['code', 'tokens'])
code_token.code = np.int64(code_token.code)

In [10]:
code2level_series = pd.read_csv("./data/code2level_series")
code2level_series = np.array(code2level_series).reshape(-1)
code2level_series

array([0, 1, 2, ..., 0, 0, 0])

In [11]:
code_token = code_token.assign(level= 0)
for i in range(len(code_token)):
    level = code2level_series[code_token['code'][i]]
    code_token.loc[i,'level'] = level
code_token.loc[45:49,:]

,code,tokens,level
45,166,"[SUPERGA, 義, 大利, 國民經典, 男女, 款休, 閒鞋, 1, 雙]",1
46,166,"[SUPERGA, 義, 大利, 國民經典, 男女, 款休, 閒鞋, 3, 雙]",1
47,536,"[MIT, 醫療級, 三層, 防護, 口罩, 久筑]",1
48,1011,"[COMFY, air, Sofa, 最新, 正版, 10, 秒, 快充, 便利, 戶外, ...",2
49,1011,"[COMFY, air, Sofa, 最新, 正版, 10, 秒, 快充, 便利, 戶外, ...",2


In [12]:
# code_token = pd.read_csv("./data/code_token_201906131152",converters={'tokens': eval})

對code,level進行groupby前，先將code為0的列清除

In [13]:
# code_token.code==0
print(code_token.shape)
tmp = pd.Series(range(len(code_token)))
tmp = list(tmp[code_token.code==0])
code_token_drop0 =code_token.drop(tmp)
code_token_drop0 = code_token_drop0.reset_index(drop=True)
print(code_token_drop0.shape)

(546216, 3)
(534746, 3)


In [14]:
code_token.tokens[4]

['aibo', '領導者', '立體聲', '智慧', '藍牙耳機', '麥克風', '系列', '8', '入', '方案']

將相同code,level的商品聚集在一起，彙整關鍵字

In [15]:
code_tokenGy = code_token_drop0.groupby(['level','code']).apply(lambda x : np.concatenate(np.array(x.tokens)))
code_tokenGy.iloc[:5]

level  code
1      166     [SUPERGA, 義, 大利, 國民經典, 男女, 款休, 閒鞋, 2, 雙, SUPER...
       222     [aibo, 領導者, 立體聲, 智慧, 藍牙耳機, 麥克風, 系列, 2, 入, 方案, ...
       436     [RICHOME, 理想, 生活, 無段, 式, 休閒, 躺椅, 瑞比大, 蝴蝶, 腰, 枕...
       536     [THERMOS, 膳, 魔師, 超輕量, 不, 鏽, 鋼, 真空, 保溫杯, 保溫, 瓶,...
       537     [美姬心, MGC, 醫美級, 私密, 保養, 噴霧, 2, 入, 贈萌, 萌貓, 零錢, ...
dtype: object

In [ ]:
計算重複關鍵字的數量

In [16]:
from collections import Counter
code_tokenGyCount = pd.Series([Counter(x) for x in code_tokenGy],index=code_tokenGy.index)
code_tokenGyCount.iloc[:5]

level  code
1      166     {'SUPERGA': 28, '義': 109, '大利': 114, '國民經典': 1...
       222     {'aibo': 391, '領導者': 40, '立體聲': 91, '智慧': 1480...
       436     {'RICHOME': 180, '理想': 21, '生活': 222, '無段': 10...
       536     {'THERMOS': 676, '膳': 828, '魔師': 833, '超輕量': 3...
       537     {'美姬心': 4, 'MGC': 2, '醫美級': 3, '私密': 3, '保養': ...
dtype: object

將文字結果數字化，以加速運算

In [17]:
from gensim.corpora.dictionary import Dictionary
dictionary = Dictionary(list(code_tokenGyCount))

In [18]:
code_tokenGybow = pd.Series([dictionary.doc2bow(x) for x in code_tokenGyCount],index=code_tokenGyCount.index)
code_tokenGybow.iloc[:5]

level  code
1      166     [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...
       222     [(1, 1), (2, 1), (3, 1), (4, 1), (14, 1), (15,...
       436     [(1, 1), (3, 1), (14, 1), (15, 1), (17, 1), (2...
       536     [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (14, ...
       537     [(14, 1), (15, 1), (61, 1), (62, 1), (97, 1), ...
dtype: object

進行Tfidf計算

In [19]:
from gensim.models.tfidfmodel import TfidfModel
tfidf = TfidfModel(code_tokenGybow)
code_tokenGyTfidf = pd.Series(tfidf[code_tokenGybow],index=code_tokenGybow.index)
code_tokenGyTfidf.iloc[:5]

level  code
1      166     [(0, 0.009348754548710753), (1, 0.004221339032...
       222     [(1, 0.0042836341692605836), (2, 0.00543793136...
       436     [(1, 0.010058174585060815), (3, 0.008382318589...
       536     [(0, 0.0063977134842708155), (1, 0.00288882519...
       537     [(14, 0.0014761038413572088), (15, 0.023058064...
dtype: object

排序Tfidf結果，並且取權重排名50%以上的關鍵字

In [20]:
code_tokenGyTfidfSort = pd.Series(map( lambda x: sorted(x,key=lambda w: w[1],reverse=True),code_tokenGyTfidf),index=code_tokenGyTfidf.index)
code_tokenGyTfidfSort.iloc[:5]

level  code
1      166     [(5, 0.013681923351486094), (6, 0.013681923351...
       222     [(7626, 0.013883830204885834), (7627, 0.013883...
       436     [(13989, 0.032599886589798885), (13992, 0.0325...
       536     [(14985, 0.009363068102866561), (14986, 0.0093...
       537     [(26687, 0.25958677325077517), (26689, 0.25958...
dtype: object

In [21]:
code_tokenGyTfidfSort2 = pd.Series(map( lambda x: x[:len(x)//2],code_tokenGyTfidfSort),index=code_tokenGyTfidfSort.index)
code_tokenGyTfidfSort2[:5]

level  code
1      166     [(5, 0.013681923351486094), (6, 0.013681923351...
       222     [(7626, 0.013883830204885834), (7627, 0.013883...
       436     [(13989, 0.032599886589798885), (13992, 0.0325...
       536     [(14985, 0.009363068102866561), (14986, 0.0093...
       537     [(26687, 0.25958677325077517), (26689, 0.25958...
dtype: object

In [22]:
code_tokenGyTfidfSort3 = pd.Series(map( lambda x: list(map( lambda y: y[0]  ,x)),code_tokenGyTfidfSort),index=code_tokenGyTfidfSort.index)
code_tokenGyTfidfSort3[:5]

level  code
1      166     [5, 6, 7, 8, 9, 10, 11, 12, 13, 25, 26, 30, 31...
       222     [7626, 7627, 7628, 7630, 7632, 7633, 7634, 763...
       436     [13989, 13992, 13994, 13995, 13996, 13998, 139...
       536     [14985, 14986, 14988, 14991, 14994, 14995, 149...
       537     [26687, 26689, 26691, 26692, 26693, 26690, 266...
dtype: object

In [23]:
code_tokenGyTfidfSort3[4][:5]

code
181    [64988, 2634, 4348, 1011, 2704, 7537, 4612, 64...
209    [64990, 64991, 64992, 64993, 64994, 64995, 649...
214    [65047, 65048, 65050, 65051, 65052, 65053, 650...
215    [65080, 65081, 65082, 65083, 65084, 65085, 650...
230    [65108, 65110, 45771, 46467, 58222, 65109, 911...
dtype: object

將code 商品名稱的tokens level newCode 建立新Dataframe

In [ ]:
code_token_new = code_token.assign(newCode= 0)
print(code_token_new.shape)
code_token_new.iloc[:5,:]

Counter計算詞數

In [25]:
code_token_new.tokens = pd.Series([Counter(x) for x in code_token.tokens])
code_token_new.iloc[:5,:]

,code,tokens,level,newCode
0,5821,"{'柯絲妃': 1, '兒胎': 1, '盤素': 1, '肌因': 1, '賦活': 1,...",4,0
1,222,"{'aibo': 1, '領導者': 1, '立體聲': 1, '智慧': 1, '藍牙耳機...",1,0
2,222,"{'aibo': 1, '領導者': 1, '立體聲': 1, '智慧': 1, '藍牙耳機...",1,0
3,222,"{'aibo': 1, '領導者': 1, '立體聲': 1, '智慧': 1, '藍牙耳機...",1,0
4,222,"{'aibo': 1, '領導者': 1, '立體聲': 1, '智慧': 1, '藍牙耳機...",1,0


數字化關鍵字

In [26]:
code_token_new.tokens = pd.Series([dictionary.doc2bow(x) for x in code_token_new.tokens])
code_token_new.iloc[:5,:]

,code,tokens,level,newCode
0,5821,"[(15, 1), (1667, 1), (6545, 1), (47241, 1), (4...",4,0
1,222,"[(61, 1), (1667, 1), (3692, 1), (3806, 1), (52...",1,0
2,222,"[(133, 1), (1667, 1), (3692, 1), (3806, 1), (5...",1,0
3,222,"[(14, 1), (1667, 1), (3692, 1), (3806, 1), (52...",1,0
4,222,"[(190, 1), (1667, 1), (3692, 1), (3806, 1), (5...",1,0


In [27]:
code_token_new.tokens = pd.Series(map( lambda x: list(map( lambda y: y[0]  ,x)),code_token_new.tokens))
code_token_new.iloc[:5,:]

,code,tokens,level,newCode
0,5821,"[15, 1667, 6545, 47241, 47492, 65824, 68045, 7...",4,0
1,222,"[61, 1667, 3692, 3806, 5229, 10358, 12629, 129...",1,0
2,222,"[133, 1667, 3692, 3806, 5229, 10358, 12629, 12...",1,0
3,222,"[14, 1667, 3692, 3806, 5229, 10358, 12629, 129...",1,0
4,222,"[190, 1667, 3692, 3806, 5229, 10358, 12629, 12...",1,0


找尋合適分類的運算

In [28]:
def ifMatchClass(itemTokens,classTokens):
    numTok = len(itemTokens)
    matchCount = 0
    for x in itemTokens:
        if x in classTokens:
            matchCount += 1
    if matchCount > numTok//2:
        return True
    else:
        return False

In [ ]:
%%time
matchClass = 6
checklevel = 3

newCode = np.zeros(len(code_token_new), dtype=np.int64)
for idxItem, row in code_token_new.iterrows():
    if row.level <= checklevel:
        itemTokens = row.tokens
        index = code_tokenGyTfidfSort3[matchClass].index
        for idxClass, x in enumerate(code_tokenGyTfidfSort3[matchClass]):
            if ifMatchClass(itemTokens,x):
                code_token_new.loc[idxItem,"newCode"]=index[idxClass]

In [ ]:
%%time
matchClass = 5
checklevel = 3

newCode = np.zeros(len(code_token_new), dtype=np.int64)
for idxItem, row in code_token_new.iterrows():
    if row.level <= checklevel:
        itemTokens = row.tokens
        index = code_tokenGyTfidfSort3[matchClass].index
        for idxClass, x in enumerate(code_tokenGyTfidfSort3[matchClass]):
            if ifMatchClass(itemTokens,x):
                code_token_new.loc[idxItem,"newCode"]=index[idxClass]

In [ ]:
%%time
matchClass = 4
checklevel = 3

newCode = np.zeros(len(code_token_new), dtype=np.int64)
for idxItem, row in code_token_new.iterrows():
    if row.level <= checklevel:
        itemTokens = row.tokens
        index = code_tokenGyTfidfSort3[matchClass].index
        for idxClass, x in enumerate(code_tokenGyTfidfSort3[matchClass]):
            if ifMatchClass(itemTokens,x):
                code_token_new.loc[idxItem,"newCode"]=index[idxClass]